In [2]:
import librosa
import librosa.feature
import os
import numpy as np
from keras.models import load_model
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.utils.np_utils import to_categorical
from keras.preprocessing import sequence
from keras import initializers


Using TensorFlow backend.


In [4]:
model = load_model("weights_spec_2.h5")

(128, 8189)


In [6]:
def splitFile(filename):
        song, sr = librosa.load(filename, mono=True, sr = 22050)
        mel = librosa.feature.melspectrogram(song, sr)
        max_len = 217
        r = int(mel.shape[1]/max_len) + 1
        specs = []
        for i in range(r):
            temp = mel[:,i*217:(i+1)*217]
            if (max_len > temp.shape[1]):
                pad_width = max_len - temp.shape[1]
                temp = np.pad(temp, pad_width=((0, 0), (0, pad_width)), mode='constant')
            temp= temp.reshape(1,temp.shape[0],temp.shape[1],1)
            specs.append(temp)
        return specs

In [13]:
def guessSong(splitSong):
    genre_decode = {
    0: "Electronic",
    1: "Experimental",
    2: "Folk",
    3: "Hip-Hop",
    4: "Instrumental",
    5: "International",
    6: "Pop",
    7: "Rock",
    }
    guesses = {}
    for part in splitSong:
        #take the 0th element since predict_proba returns a list with a list inside of it
        probs = model.predict_proba(part)[0]
        confidence = 0
        for prob in probs:
            if prob > confidence:
                confidence = prob
        if confidence < .5:
            print("Slice rejected due to low confidence in vote")
        else:
            
            guess = model.predict_classes(part)
            genre_name = genre_decode[guess[0]]
            if genre_name in guesses:
                guesses[genre_name] = guesses[genre_name] + 1
            else:
                guesses[genre_name] = 1
    max_guesses = 0
    prediction = ""
    for key in guesses:
        if guesses[key] > max_guesses:
            prediction = key
            max_guesses = guesses[key]
    return prediction

In [18]:
files = os.listdir("Database/ClassicalTest/")
numFiles = len(files)
rightGuesses = 0
wrongGuesses = []
for filename in files:
    filename = "Database/ClassicalTest/"+filename
    split = splitFile(filename)
    prediction = guessSong(split)
    print("I think that " + filename[17:] + " is " + prediction)
    if prediction == "Instrumental":
        rightGuesses = rightGuesses+1
    else:
        wrongGuesses.append(filename + ":" + prediction)

accuracy = rightGuesses/numFiles * 100
print("The model was " + str(accuracy) + "% accurate at classifying Instrumental songs")

Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
I think that lTest/06 Gershwin_ Piano Concerto In F - 3.mp3 is Instrumental
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
I think that lTest/3-02 Beethoven_ Symphony #5 In C Minor, Op. 67 - 2. Andante Con Moto.mp3 is Experimental
Slice rejected due to low confidence in vote
Slice rejected due to low confidence in vote
Slice 

In [19]:
for string in wrongGuesses:
    print(string)

Database/ClassicalTest/3-02 Beethoven_ Symphony #5 In C Minor, Op. 67 - 2. Andante Con Moto.mp3:Experimental
Database/ClassicalTest/Beethoven 5, 3rd and 4th mvts.mp3:Experimental
Database/ClassicalTest/01 Tchaikovsky_ Piano Concerto #1 In.mp3:Experimental
Database/ClassicalTest/02 Gershwin_ An American In Paris -.mp3:Experimental
Database/ClassicalTest/05 Gershwin_ Piano Concerto In F - 2.mp3:Experimental
Database/ClassicalTest/02 Tchaikovsky_ Piano Concerto #1 In.mp3:Experimental
